# 2D Monitoring Element Exploration

This notebook is intended to showcase `dqmexplore`'s 2D monitoring element (ME) exploration tools.

## Setup

If you are working in SWAN, you will need to install the `omsapi` as well as `dqmexplore` itself.

In [ ]:
# Run this if you are in SWAN
# Package installation/updating
!pip3 install -e .. --no-dependencies
!pip3 install omsapi
!pip3 install cmsdials --upgrade

### Imports

In [ ]:
# DIALS API
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)
dials = Dials(creds)

Run your prefered authentication method for the OMS API. For more information on the OMS API, please visit https://gitlab.cern.ch/cmsoms/oms-api-client

In [ ]:
# Method 1: krb
import omsapi

oms_fetch = omsapi.OMSAPI("https://cmsoms.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_krb()

In [ ]:
# Method 2: oidc
import omsapi
import json

with open("./clientid.json", "r") as file:
    secrets = json.load(file)

oms_fetch = omsapi.OMSAPI("http://vocms0185.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_oidc(secrets["API_CLIENT_ID"], secrets["API_CLIENT_SECRET"], audience="cmsoms-int-0185")

del(secrets)
del(file)

In [ ]:
# DQMExplore
import dqmexplore
from dqmexplore.plotting import plot2DMEs
from dqmexplore.omsutils import get_rate, plot_rate

# Plotly
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

# Other useful libraries for data exploration, manipulation and processing
import numpy as np
import pandas as pd

If you run the following, you will get a list of all the available 1D MEs.

In [ ]:
mes_df = pd.DataFrame([me_qry_rslt.__dict__ for me_qry_rslt in dials.mes.list()])
mes_df[mes_df["dim"] == 2]

## Using OMS to Obtain Metadata

Using the OMS API, we can access important information regarding the run conditions and other information about the run. The available endpoints are:

* `lumisections`
* `runs`
* `fills`
* `datasetrates`

You can access the trigger rate* in the following way:

*HLT ZeroBias trigger rate

In [ ]:
runnb = 380238
omstrig_df = get_rate(oms_fetch, runnb, "ZeroBias")
fig = plot_rate(omstrig_df, norm=False, show=True)

# Optionally, you can export the Plotly figure object dqmexplore returns. Just remember to set show=False in plot_rate
# plot(fig, filename=f"./plots/trigrate_{runnb}.html")

## 2D Monitoring Elements

2D monitoring elements are also available in DIALS and they can be accessed as shown below.

In [ ]:
runnb = 378981
me__regex = "PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_."
# me__regex = "(PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_(1|2|3))|(PixelPhase1/Tracks/clusterposition_zphi_ontrack)"
# me__regex = "PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_+."

data2D = dials.h2d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Getting trigger rate
trig_rate = get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

In [9]:
# Defining plot features
ax_labels = [
    dict(x="SignedModuleCoord", y="SignedLadderCoord")
] * 4

fig_title = f"Pixel Barrel Digi Occupancy Normalized by Trigger Rate (Run {runnb})"

fig2D = plot2DMEs(
    data2D,
    fig_title=fig_title,
    ax_labels=ax_labels,
    trigger_rates=trig_rate,
    height=500,
    width=500,
    show=True
)

# plot(fig2D, filename=f"PixelBarrelDigiOccupancy-run{runnb}-trignorm.html"))
# fig2D.write_html("plot.html", include_plotlyjs="cdn")